In [80]:
class EnigmaConfig:
    enigma_data = {
        'Alphabet': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'],
        'Notches': [('I', 'Q'), ('II', 'E'), ('III', 'V'), ('IV', 'J'), ('V', 'Z'), ('Beta', 'None'), ('Gamma', 'None')],
        'Beta': ['L', 'E', 'Y', 'J', 'V', 'C', 'N', 'I', 'X', 'W', 'P', 'B', 'Q', 'M', 'D', 'R', 'T', 'A', 'K', 'Z', 'G', 'F', 'U', 'H', 'O', 'S'],
        'Gamma': ['F', 'S', 'O', 'K', 'A', 'N', 'U', 'E', 'R', 'H', 'M', 'B', 'T', 'I', 'Y', 'C', 'W', 'L', 'Q', 'P', 'Z', 'X', 'V', 'G', 'J', 'D'],
        'I': ['E', 'K', 'M', 'F', 'L', 'G', 'D', 'Q', 'V', 'Z', 'N', 'T', 'O', 'W', 'Y', 'H', 'X', 'U', 'S', 'P', 'A', 'I', 'B', 'R', 'C', 'J'],
        'II': ['A', 'J', 'D', 'K', 'S', 'I', 'R', 'U', 'X', 'B', 'L', 'H', 'W', 'T', 'M', 'C', 'Q', 'G', 'Z', 'N', 'P', 'Y', 'F', 'V', 'O', 'E'],
        'III': ['B', 'D', 'F', 'H', 'J', 'L', 'C', 'P', 'R', 'T', 'X', 'V', 'Z', 'N', 'Y', 'E', 'I', 'W', 'G', 'A', 'K', 'M', 'U', 'S', 'Q', 'O'],
        'IV': ['E', 'S', 'O', 'V', 'P', 'Z', 'J', 'A', 'Y', 'Q', 'U', 'I', 'R', 'H', 'X', 'L', 'N', 'F', 'T', 'G', 'K', 'D', 'C', 'M', 'W', 'B'],
        'V': ['V', 'Z', 'B', 'R', 'G', 'I', 'T', 'Y', 'U', 'P', 'S', 'D', 'N', 'H', 'L', 'X', 'A', 'W', 'M', 'J', 'Q', 'O', 'F', 'E', 'C', 'K'],
        'A': ['E', 'J', 'M', 'Z', 'A', 'L', 'Y', 'X', 'V', 'B', 'W', 'F', 'C', 'R', 'Q', 'U', 'O', 'N', 'T', 'S', 'P', 'I', 'K', 'H', 'G', 'D'],
        'B': ['Y', 'R', 'U', 'H', 'Q', 'S', 'L', 'D', 'P', 'X', 'N', 'G', 'O', 'K', 'M', 'I', 'E', 'B', 'F', 'Z', 'C','W', 'V', 'J', 'A', 'T'],
        'C': ['F', 'V', 'P', 'J', 'I', 'A', 'O', 'Y', 'E', 'D', 'R', 'Z', 'X', 'W', 'G', 'C', 'T', 'K', 'U', 'Q', 'S', 'B', 'N', 'M', 'H', 'L']}
    
 

class PlugLead:
    def __init__(self, character_map):
        self.character_map = character_map
        self.character_1 = list(character_map)[0]
        self.character_2 = list(character_map)[1]

    def encode(self, character):
        if character == self.character_1:
            return self.character_2
        elif character == self.character_2:
            return self.character_1
        return character


 
class Plugboard:
    def __init__(self):
        self.leads = []

    def add(self, plug_lead):
        self.leads.append(plug_lead)

    def encode(self, character):
        for lead in self.leads:
            encoded_character = lead.encode(character)
            if character != encoded_character:
                return encoded_character
        return character

    def delete(self):
        self.leads = self.leads[:-2]
        

 
 
class Rotor_Adjuster():
    def __init__(self, ring_settings):
        self.ring_settings = ring_settings

    def rotor_offsetter(self, position_offset):
        alphabet = EnigmaConfig.enigma_data['Alphabet']
        rotor_order = [alphabet[(i + position_offset) % 26] for i in range(26)]
        return rotor_order

    def rotor_transition(self, rotor_transition_offset, char):
        alphabet_idx = EnigmaConfig.enigma_data['Alphabet'].index(char)
        transitioned_char = EnigmaConfig.enigma_data['Alphabet'][(alphabet_idx + rotor_transition_offset) % 26]
        return transitioned_char

    def ring_offsetter(self, position_settings, reverse=False):
        offset_position_settings = []
        settings_list = [(self.ring_settings[i] - 1, position) for i, position in enumerate(position_settings)]
        if reverse:
            self.ring_settings = [26 - setting + 2 for setting in self.ring_settings]
        for setting, position in settings_list:
            if position != 'None':
                character_idx = EnigmaConfig.enigma_data['Alphabet'].index(position)
                offset_idx = (character_idx - setting) % 26
                offset_character = EnigmaConfig.enigma_data['Alphabet'][offset_idx]
                offset_position_settings.append(offset_character)
            else:
                offset_position_settings.append(position)
        return offset_position_settings

    def set_ring_settings(self, ring_settings):
        self.ring_settings = ring_settings




class Notch_Adjuster():
    def __init__(self, ring_settings, rotor_engine, rotor_adjuster):
        self.ring_settings = ring_settings
        self.rotor_engine = rotor_engine
        self.rotor_adjuster = rotor_adjuster

    def adjust(self, rotor_start_positions):
        rotor_positions = [position[0] for position in rotor_start_positions]
        position_notches = []
        unadjusted_notches = []
        for rotor in self.rotor_engine.get_rotors():
            for notch in EnigmaConfig.enigma_data['Notches']:
                if notch[0] == rotor.get_rotor_type():
                    position_notches.append(notch[0])
                    unadjusted_notches.append(notch[1])
        adjusted_notches = self.rotor_adjuster.ring_offsetter(unadjusted_notches, False)
        final_position_notches = list(zip(position_notches, adjusted_notches))
        return final_position_notches, rotor_positions

    def set_ring_settings(self, ring_settings):
        self.ring_settings = ring_settings



class Rotor_Engine:
    def __init__(self, rotor_adjuster, reflector):
        self.rotors = []
        self.rotor_adjuster = rotor_adjuster
        self.reflector = reflector

    def encode_controller(self, character_in, rotor_positions):
        rotor_positions.reverse()
        rotor_offsets = []
        for i in rotor_positions:
            rotor_offsets.append(EnigmaConfig.enigma_data['Alphabet'].index(i[0]))
        encoded_char = self.encoder(rotor_positions, rotor_offsets, 0, None, character_in)
        encoded_char = self.encoder(rotor_positions, rotor_offsets, 1, 'Forward', encoded_char)
        encoded_char = self.encoder(rotor_positions, rotor_offsets, 2, 'Forward', encoded_char)
        if self.get_rotors_count() == 4:
            encoded_char = self.encoder(rotor_positions, rotor_offsets, 3, 'Forward', encoded_char)
        encoded_char = self.reflector.encode(rotor_offsets, encoded_char)
        if self.get_rotors_count() == 4:
            encoded_char = self.encoder(rotor_positions, rotor_offsets, 3, 'Reflector', encoded_char)
        if self.get_rotors_count() == 4:
            encoded_char = self.encoder(rotor_positions, rotor_offsets, 2, 'Backward', encoded_char)
        else:
            encoded_char = self.encoder(rotor_positions, rotor_offsets, 2, 'Reflector', encoded_char)
        encoded_char = self.encoder(rotor_positions, rotor_offsets, 1, 'Backward', encoded_char)
        encoded_char = self.encoder(rotor_positions, rotor_offsets, 0, 'Backward', encoded_char)
        output_char = EnigmaConfig.enigma_data['Alphabet'][(EnigmaConfig.enigma_data['Alphabet'].index(encoded_char) - EnigmaConfig.enigma_data['Alphabet'].index(rotor_positions[0][0])) % 26]
        return output_char

    def encoder(self, rotor_positions, rotor_offsets, idx, rotor_transition_offset, input_char):
        rotor_position = rotor_positions[idx]
        if rotor_transition_offset == None:
            rotor_char_idx = EnigmaConfig.enigma_data['Alphabet'].index(input_char)
            rotor_char = rotor_position[rotor_char_idx]
        elif rotor_transition_offset == 'Reflector':
            rotor_char = self.rotor_adjuster.rotor_transition(rotor_offsets[idx], input_char)
        elif rotor_transition_offset == 'Forward':
            rotor_char = self.rotor_adjuster.rotor_transition(rotor_offsets[idx] - rotor_offsets[idx - 1], input_char)
        else:
            if idx == 3:
                rotor_char = self.rotor_adjuster.rotor_transition(rotor_offsets[idx], input_char)
            else:
                rotor_char = self.rotor_adjuster.rotor_transition(rotor_offsets[idx] - rotor_offsets[idx + 1], input_char)
        if rotor_transition_offset == 'Forward' or rotor_transition_offset == None:
            return self.rotors[::-1][idx].encode_right_to_left(rotor_char)
        else:
            return self.rotors[::-1][idx].encode_left_to_right(rotor_char)

    def add_rotor(self, rotor):
        self.rotors.append(rotor)

    def get_rotors_count(self):
        return len(self.rotors)

    def get_rotors(self):
        return self.rotors

    def reset_rotors(self):
        self.rotors = []

    def set_reflector(self, reflector):
        self.reflector = reflector



class Rotor:
    def __init__(self, rotor_type):
        self.rotor_type = rotor_type

    def encode_right_to_left(self, character):
        char_idx = EnigmaConfig.enigma_data['Alphabet'].index(character)
        return EnigmaConfig.enigma_data[self.rotor_type][char_idx]

    def encode_left_to_right(self, character):
        char_idx = EnigmaConfig.enigma_data[self.rotor_type].index(character)
        return EnigmaConfig.enigma_data['Alphabet'][char_idx]

    def get_rotor_type(self):
        return self.rotor_type



class Reflector:
    def __init__(self, reflector_type, rotor_adjuster):
        self.reflector_type = reflector_type
        self.rotor_count = 0
        self.rotor_adjuster = rotor_adjuster
 
    def encode(self, rotor_offsets, encoded_char):
        if self.rotor_count == 4:
            output_char = self.rotor_adjuster.rotor_transition(0 - rotor_offsets[3], encoded_char)
        else:
            output_char = self.rotor_adjuster.rotor_transition(0 - rotor_offsets[2], encoded_char)
        if type(self.reflector_type) == list:
            output_char = self.reflector_type[EnigmaConfig.enigma_data['Alphabet'].index(output_char)]
        else:
            output_char = EnigmaConfig.enigma_data[self.reflector_type][EnigmaConfig.enigma_data['Alphabet'].index(output_char)]
        return output_char

    def set_rotor_count(self, count):
        self.rotor_count = count 

    def set_reflector_type(self, reflector_type):
        self.reflector_type = reflector_type 
 

class EnigmaMachine:
    def __init__(self, position_settings, rotor_engine, rotor_turner, notch_adjuster, rotor_adjuster, reflector, plugboard):
        self.position_settings = position_settings
        self.rotor_engine = rotor_engine
        self.rotor_turner = rotor_turner
        self.notch_adjuster = notch_adjuster
        self.rotor_adjuster = rotor_adjuster
        self.reflector = reflector
        self.plugboard = plugboard

    def encode(self, message, plugboard_switch):
        encoded_message = ''
        for character in message:
            if character == ' ':
                encoded_message += ' '
            else:
                if plugboard_switch == True:
                    character = self.plugboard.encode(character)
                adjusted_position_settings = self.rotor_adjuster.ring_offsetter(self.position_settings, False)
                rotor_start_positions = []
                for i in adjusted_position_settings:
                    rotor = self.rotor_adjuster.rotor_offsetter(EnigmaConfig.enigma_data['Alphabet'].index(i))
                    rotor_start_positions.append(rotor)
                notch_data = self.notch_adjuster.adjust(rotor_start_positions)
                rotor_positions = self.rotor_turner.turn(notch_data[0], notch_data[1])
                character = self.rotor_engine.encode_controller(character, rotor_positions[0])
                if plugboard_switch == True:
                    character = self.plugboard.encode(character)
                self.position_settings = rotor_positions[1]
                encoded_message += character
        return encoded_message

    def reset_position_settings(self, position_settings):
        self.position_settings = position_settings

    def set_reflector(self, reflector):
        self.reflector = reflector



plugboard = Plugboard()
plugboard.add(PlugLead('HL'))
plugboard.add(PlugLead('MO'))
plugboard.add(PlugLead('AJ'))
plugboard.add(PlugLead('CX'))

ring_settings = [1, 1, 1]
rotor_adjuster = Rotor_Adjuster(ring_settings)
reflector = Reflector('B', rotor_adjuster)
rotor_engine = Rotor_Engine(rotor_adjuster, reflector)
rotor_engine.add_rotor(Rotor('I'))
rotor_engine.add_rotor(Rotor('II'))
rotor_engine.add_rotor(Rotor('III'))
reflector.set_rotor_count(rotor_engine.get_rotors_count())

rotor_turner = Rotor_Turner(ring_settings, rotor_adjuster)
notch_adjuster = Notch_Adjuster(ring_settings, rotor_engine, rotor_adjuster)

enigma = EnigmaMachine(['A', 'A', 'Z'], rotor_engine, rotor_turner, notch_adjuster, rotor_adjuster, reflector, plugboard)
enigma.encode('HELLOWORLD', True)

'SFPTMQXNVR'

In [ ]:
SFPTMQXNVR

In [110]:
# Code Crack 5

import itertools

def create_reflector_mapping(reflector_list):
    reflector_mapping = {}
    for index, letter in enumerate(reflector_list):
        reflector_mapping[chr(65 + index)] = letter
    return reflector_mapping

def swap_reflector_pairs(reflector, pair1, pair2):
    letter1, letter2 = pair1
    letter3, letter4 = pair2
    reflector[letter1], reflector[letter3] = reflector[letter3], reflector[letter1]
    reflector[letter2], reflector[letter4] = reflector[letter4], reflector[letter2]

def generate_all_reflector_swaps(original_reflector):
    letters = list(original_reflector.keys())
    all_pairs = list(itertools.combinations(letters, 2))
    pair_combinations = itertools.combinations(all_pairs, 2)
    all_possible_reflectors = []
    for pair_combo in pair_combinations:
        pair1, pair2 = pair_combo
        unique_letters = set(pair1 + pair2)
        if len(unique_letters) == 4:
            reflector_copy = original_reflector.copy()
            swap_reflector_pairs(reflector_copy, pair1, pair2)
            all_possible_reflectors.append(reflector_copy)
    return all_possible_reflectors

def reflector_to_list(reflector_dict):
    sorted_items = sorted(reflector_dict.items())  
    return [letter for _, letter in sorted_items]


plugboard_5 = Plugboard()
plugboard_5.add(PlugLead('UG'))
plugboard_5.add(PlugLead('IE'))
plugboard_5.add(PlugLead('PO'))
plugboard_5.add(PlugLead('NX'))
plugboard_5.add(PlugLead('WT'))

ring_settings_5 = [6, 18, 7]
rotor_adjuster_5 = Rotor_Adjuster(ring_settings_5)
reflector_5 = Reflector('A', rotor_adjuster_5)
rotor_engine_5 = Rotor_Engine(rotor_adjuster_5, reflector_5)
rotor_engine_5.add_rotor(Rotor('V'))
rotor_engine_5.add_rotor(Rotor('II'))
rotor_engine_5.add_rotor(Rotor('IV'))
reflector_5.set_rotor_count(rotor_engine_5.get_rotors_count())
rotor_turner_5 = Rotor_Turner(ring_settings_5, rotor_adjuster_5)
notch_adjuster_5 = Notch_Adjuster(ring_settings_5, rotor_engine_5, rotor_adjuster_5)
enigma_5 = EnigmaMachine(['A', 'J', 'L'], rotor_engine_5, rotor_turner_5, notch_adjuster_5, rotor_adjuster_5, reflector_5, plugboard_5)


encoded_message = 'HWREISXLGTTBYVXRCWWJAKZDTVZWKBDJPVQYNEQIOTIFX'
cribs = ['YOU', 'CAN', 'INSTA', 'GRAM', 'FACEBOOK', 'INSTAGRAM', 'LINKEDIN', 'TWITTER', 'SNAPCHAT', 'WECHAT', 'MYSPACE', 'BEBO', 'TIKTOK', 'REDDIT', 'PINTEREST', 'YOUTUBE', 'WHATSAPP', 'TUMBLR', 'TELEGRAM', 'VIBER', 'LINE', 'BAIDU', 'SINA', 'QZONE', 'DOUBAN', 'TWITCH', 'SIGNAL', 'BUMBLE', 'DISCORD', 'HINGE', 'QUORA', 'FLICKR', 'NEXTDOOR', 'SIGNAL', 'MESSENGER', 'DOUYIN', 'KUAISHOU', 'WEIBOU', 'TEAMS', 'TIEBA', 'SKYPE']
# cribs = 'AND', 'WITH', 'HAVE', 'THIS', 'THAT', 'THE', 'WAS', 'FOR', 'THESE', 'YOUVE', 'ARE',
base_reflectors = [EnigmaConfig.enigma_data['A'], EnigmaConfig.enigma_data['B'] , EnigmaConfig.enigma_data['C']]


def decode_5(encoded_message, base_reflector, cribs):
    total_count = 0
    run_count = 0
    for base_reflector in base_reflectors:
        print('Base Reflector')
        original_reflector = create_reflector_mapping(base_reflector)
        all_possible_reflectors = generate_all_reflector_swaps(original_reflector)
        all_possible_reflectors_lists = [reflector_to_list(reflector) for reflector in all_possible_reflectors]
        for modified_reflector in all_possible_reflectors_lists:
            reflector_5.set_reflector_type(modified_reflector)
            enigma_5.set_reflector(reflector_5)
            rotor_engine_5.set_reflector(reflector_5)
            decoded_message = enigma_5.encode(encoded_message, True)
            for crib in cribs:
                if crib in decoded_message and decoded_message in 'INSTA' and decoded_message in 'YOU':
                    print(decoded_message)
                    print(modified_reflector)
                    print()
            run_count += 1
            if run_count == 10000:
                total_count += run_count
                print(total_count)
                run_count = 0

decode_5(encoded_message, base_reflector, cribs)

Base Reflector


KeyboardInterrupt: 

In [ ]:
HWREISXLGTTBYVXRCWWJAKZDTVZWKBDJPVQYNEQIOTIFX

In [ ]:
IHGJCJGTLUZYPAWOGCANOWWEYOUUAXUOVAXAKXIRXNGSG

In [ ]:
['F', 'V', 'P', 'A', 'I', 'J', 'O', 'Y', 'M', 'D', 'R', 'Z', 'X', 'W', 'G', 'C', 'T', 'K', 'U', 'Q', 'S', 'B', 'N', 'E', 'H', 'L']